<a href="https://colab.research.google.com/github/LaylaMcD/DIGS-DP/blob/main/Word2Vec_Pt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports and set up logging
import gensim
import logging
import glob, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# directory containing all source texts for training the model
data_dir="/content/drive/My Drive/Colab Notebooks/output_files"

In [4]:
os.chdir(data_dir)
documents = list()
for filename in glob.glob("*.txt"):
    filedata = open(filename, 'r').read()
    print(filename + " = " + str(len(filedata)) + " chars")
    documents = documents + filedata.split(".")

chunk_6.txt = 2625232 chars
chunk_4.txt = 2643870 chars
chunk_7.txt = 3051987 chars
chunk_2.txt = 2889349 chars
chunk_8.txt = 2336498 chars
chunk_3.txt = 2559872 chars
chunk_1.txt = 2792133 chars
chunk_5.txt = 2774493 chars
chunk_10.txt = 2428297 chars
chunk_9.txt = 2417660 chars


In [6]:
#is that first sentence correct?
print(documents[0])

docs2 = []
# remove all the "\n"s
for doc in documents:
    newdoc = doc.replace("\n", " ")
    docs2.append(newdoc)

print(docs2[0])

"Twitter is now turning on its new algorithmic timeline for everyone
, with users reporting that the company started turning it on it across the service as early as March 15
"Twitter is now turning on its new algorithmic timeline for everyone , with users reporting that the company started turning it on it across the service as early as March 15


In [7]:
from gensim.models import word2vec, Phrases
from gensim.models.phrases import Phraser

#documents = ["the mayor of new york was there", "human computer interaction and machine learning has now become a trending research area","human computer interaction is interesting","human computer interaction is a pretty interesting subject", "human computer interaction is a great and new subject", "machine learning can be useful sometimes","new york mayor was present", "I love machine learning because it is a new subject area", "human computer interaction helps people to get user friendly applications"]

sentence_stream = [doc.split(" ") for doc in docs2]  #documents

trigram_sentences_project = []

bigram = Phraser(Phrases(sentence_stream))
trigram = Phraser(Phrases(bigram[sentence_stream]))

for sent in sentence_stream:
    bigrams_ = bigram[sent]
    trigrams_ = trigram[bigram[sent]]
    trigram_sentences_project.append(trigrams_)

# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 1    # Minimum word count
num_workers = 20      # Number of threads to run in parallel
context = 8           # Context window size
downsampling = 1e-3   # Downsample setting for frequent words
skip_grams = 1        # 0 for CBOW, 1 for skip-grams

model = word2vec.Word2Vec(trigram_sentences_project, workers=num_workers, \
           min_count = min_word_count, \
           window = context, sample = downsampling, sg = skip_grams)


vocab = list(model.wv.index_to_key)
print(vocab[:10])

['the', 'and', 'of', 'to', 'a', 'in', 'is', 'for', 'that', '-']


In [8]:
# Print the total number of items in our model's vocabulary
print(len(model.wv.index_to_key))

331301


In [10]:
w1 = "lady"
model.wv.most_similar (positive=w1)

[('girlfriend', 0.9164406061172485),
 ('mom', 0.9089912176132202),
 ('ghost', 0.9057749509811401),
 ('kid', 0.9052375555038452),
 ('an_old', 0.9045943021774292),
 ('lover', 0.9038964509963989),
 ('boy', 0.9038270115852356),
 ('sexy', 0.9033171534538269),
 ('boyfriend', 0.8983889222145081),
 ('stranger', 0.8977368474006653)]

"ghost" is certainly interesting...

In [11]:
w1 = "king"
model.wv.most_similar (positive=w1)

[('queen', 0.9430097937583923),
 ('easter_decorations_for_sale', 0.9366862177848816),
 ('centerpieces', 0.9351146221160889),
 ('toddler', 0.9335649609565735),
 ('home_decor', 0.930773913860321),
 ('bedroom_ideas', 0.9292471408843994),
 ('decorating_ideas', 0.9287014007568359),
 ('easter_decorations_ebay', 0.9242209792137146),
 ('bedroom_floor_plan', 0.9234209656715393),
 ('arc_floor_lamp', 0.9233357310295105)]

In [12]:
w1 = "prince"
model.wv.most_similar (positive=w1)

[('gentleman', 0.9449213743209839),
 ('stern', 0.9419693350791931),
 ('hymn', 0.9416805505752563),
 ('reciting', 0.9405601620674133),
 ('mythological', 0.9399741888046265),
 ('Coltrane’s', 0.9396284818649292),
 ('15-year-old', 0.9385067224502563),
 ('Jericho', 0.9383893013000488),
 ('darkly', 0.9370554685592651),
 ('godmother', 0.9369605779647827)]

In [13]:
w1 = "death"
model.wv.most_similar (positive=w1)

[('murder', 0.8344448804855347),
 ('death,', 0.8342791795730591),
 ('innocent', 0.8179368376731873),
 ('victim', 0.8175161480903625),
 ('violent', 0.8161144256591797),
 ('killing', 0.8105068206787109),
 ('assault', 0.8075164556503296),
 ('birth', 0.8075002431869507),
 ('threatening', 0.8050549030303955),
 ('guilty', 0.8026435971260071)]

In [14]:
result = model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

most_similar_key, similarity = result[0]  # look at the first match

print(f"{most_similar_key}: {similarity:.4f}")

nursery: 0.8705


In [15]:
# Use a different similarity measure: "cosmul".
result = model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = result[0]  # look at the first match
print(f"{most_similar_key}: {similarity:.4f}")


nursery: 1.0201


What does this increase in numbers mean?

In [16]:
model.wv.most_similar(positive=['he','dies'],negative=['she'])

[('old_man', 0.7500130534172058),
 ('sword,', 0.7496705651283264),
 ('teammate', 0.7471629977226257),
 ('Gardner', 0.7449076175689697),
 ('grandson', 0.7411761283874512),
 ('Jeter', 0.7382733225822449),
 ('McShane', 0.737956166267395),
 ('spear', 0.7377640604972839),
 ('third_straight', 0.7374150156974792),
 ('shook_his', 0.737359881401062)]

In [17]:
model.predict_output_word(["I","die","again"], topn=30)

[('if', 7.627527e-05),
 ('never', 6.702845e-05),
 ('I', 6.0757644e-05),
 ('but', 6.0132068e-05),
 ('would', 5.751547e-05),
 ('will', 5.6714973e-05),
 ('do', 5.524472e-05),
 ('so', 5.208102e-05),
 ('should', 5.1377625e-05),
 ('want_to', 5.0093364e-05),
 ('went_back', 4.998591e-05),
 ('you', 4.990742e-05),
 ('until', 4.909504e-05),
 ('told_him', 4.8031186e-05),
 ('my_husband', 4.744889e-05),
 ('again', 4.7268513e-05),
 ('my', 4.6656456e-05),
 ('could_see', 4.648825e-05),
 ('see', 4.5757624e-05),
 ('just', 4.5348024e-05),
 ('then', 4.461459e-05),
 ('my_hair', 4.424545e-05),
 ("couldn't", 4.4185635e-05),
 ('looked_at', 4.397541e-05),
 ('it', 4.375113e-05),
 ('them', 4.3705662e-05),
 ('eat', 4.3302727e-05),
 ('let', 4.3300024e-05),
 ('it,_but', 4.328994e-05),
 ('it,', 4.2643238e-05)]

In [22]:
import numpy as np

tensorsfp = "/content/drive/My Drive/Colab Notebooks/output_files/tensorsfp.txt"
metadatafp = "/content/drive/My Drive/Colab Notebooks/output_files/metadatafp.txt"

with open(tensorsfp, 'w+') as tensors:
    with open(metadatafp, 'w+') as metadata:
        for word in model.wv.index_to_key:
            metadata.write(word + '\n')
            vector_row = model.wv.get_vector(word)

            # Convert the numpy array to a string
            vector_row_str = np.array2string(vector_row, separator='\t')

            tensors.write(vector_row_str + '\n')